## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import config from another folder
import sys
sys.path.append('../')

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Dunedin,NZ,-45.8742,170.5036,59.81,89,79,5.99,broken clouds,4/20/2022 18:42
1,1,Chokurdakh,RU,70.6333,147.9167,2.62,99,93,4.07,overcast clouds,4/20/2022 18:42
2,2,Ponta Do Sol,PT,32.6667,-17.1000,63.23,54,13,7.20,few clouds,4/20/2022 18:42
3,3,Port Elizabeth,ZA,-33.9180,25.5701,70.11,83,0,3.44,clear sky,4/20/2022 18:41
4,4,Panalingaan,PH,8.7842,117.4231,77.58,86,33,3.11,scattered clouds,4/20/2022 18:42


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
user_cities = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
user_cities.count()

City_ID                147
City                   147
Country                144
Lat                    147
Lng                    147
Max Temp               147
Humidity               147
Cloudiness             147
Wind Speed             147
Current Description    147
Date                   147
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = user_cities.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.NaN
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Ponta Do Sol,PT,63.23,few clouds,32.6667,-17.1000,NaN
3,Port Elizabeth,ZA,70.11,clear sky,-33.9180,25.5701,NaN
17,Sinjar,IQ,64.36,clear sky,36.3209,41.8754,NaN
24,San Jose,US,70.00,scattered clouds,37.3394,-121.8950,NaN
29,Victoria,HK,73.18,overcast clouds,22.2855,114.1577,NaN
30,Carnarvon,AU,66.27,overcast clouds,-24.8667,113.6333,NaN
32,Bayan,CN,62.87,overcast clouds,46.0833,127.4000,NaN
33,Presidencia Roque Saenz Pena,AR,74.91,overcast clouds,-26.7852,-60.4388,NaN
38,Nouadhibou,MR,69.78,sand,20.9310,-17.0347,NaN
41,Newport,GB,61.68,broken clouds,51.5877,-2.9984,NaN


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and retrieve the JSON data from the search. 
    params["location"] = f"{lat},{lng}"
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Ponta Do Sol,PT,63.23,few clouds,32.6667,-17.1000,Hotel do Campo
3,Port Elizabeth,ZA,70.11,clear sky,-33.9180,25.5701,39 On Nile Guest House
17,Sinjar,IQ,64.36,clear sky,36.3209,41.8754,زوماني فون 1
24,San Jose,US,70.00,scattered clouds,37.3394,-121.8950,The Westin San Jose
29,Victoria,HK,73.18,overcast clouds,22.2855,114.1577,Mini Hotel Central
30,Carnarvon,AU,66.27,overcast clouds,-24.8667,113.6333,Hospitality Carnarvon
32,Bayan,CN,62.87,overcast clouds,46.0833,127.4000,Wanhao Inn
33,Presidencia Roque Saenz Pena,AR,74.91,overcast clouds,-26.7852,-60.4388,Hotel Gualok
38,Nouadhibou,MR,69.78,sand,20.9310,-17.0347,El Medina
41,Newport,GB,61.68,broken clouds,51.5877,-2.9984,ibis budget Newport


In [10]:
# Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
     <dl>
        <dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
        <dt><b>City</b></dt><dd>{City}</dd>
        <dt><b>Country</b></dt><dd>{Country}</dd>
        <dt><b>Weather description</b></dt><dd>{Current Description} and max {Max Temp} °F</dd>
        </dl>
"""

In [13]:
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]


In [14]:
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
list_locations = list(locations.to_records(index=False))

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(list_locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))